In [1]:
import pandas as pd

# What fraction of boulder's indigenous residents are unhoused?
* To make this calculation, we use:
    * 2019 [Census Bureau estimates](https://www.census.gov/quickfacts/fact/table/bouldercountycolorado,bouldercitycolorado/RHI825219#RHI825219) for the City of Boulder's American Indian and Alaska Native (AIAN) population
    * 2019 and 2020 data from [Metro Denver Homeless Initiative's Point in Time Survey](https://www.mdhi.org/pit). Point in Time data that is avaliable online is at the granularity of Boulder county. We requested and were provided with survey data specifically for the City of Boulder for 2019 and 2020, for fair comparion with City of Boulder's Census Bureau estimates. Point in Time Survey is a known undercount, so our calculation is a lower bound on the proportion of unhoused indigenous residents. 
    * Please write to us with any questions about this data, or our analysis. 
       - Sam Becker (samhbecker@gmail.com)
       - Nick LaBerge (labergenick@gmail.com). 

#### First, we distribute "missing" race responses from the PiT survey across races, in proportion to each group's demographic representation
* "missing" responses only occur in 2019

In [2]:
df_census = pd.read_excel('../data/boulder_city_census.xlsx').drop(columns = ['source','coverage'])
df_pit = pd.read_excel('../data/boulder_city_pit.xlsx').drop(columns = ['source','coverage'])

df_pit = df_pit.set_index('year')

for year in [2019, 2020]:
    pit_year = df_pit.loc[(df_pit.index == year)]
    num_missing = df_pit.loc[(df_pit.index == year) & (df_pit.race == 'Missing'), 'population'].iloc[0]
    print(num_missing, 'missing responses in the', year,'PiT survey.')

    #calculate the fraction of unhoused people that each group represents, excuding the people with missing race
    pit_year_pop_not_missing = pit_year.population.sum() - num_missing
    df_pit.loc[(df_pit.index == year),'frac'] = pit_year.population / pit_year_pop_not_missing

    #distribute the missing race people across races, in proportion to each group's demographic representation
    pit_year = df_pit.loc[(df_pit.index == year)]
    df_pit.loc[(df_pit.index == year),'population'] += pit_year.frac * num_missing

# remove the missing rows, now that we have distributed them across observed races
df_pit = df_pit[df_pit['race']!='Missing']

22 missing responses in the 2019 PiT survey.
0.0 missing responses in the 2020 PiT survey.


#### This gives us a lower bound on the number of boulder's unhoused indigenous residents for both years: 
* at least 33 in 2019
* at least 21 in 2020

In [3]:
df_pit[df_pit.race == 'AIAN']

,race,population,type,frac
year,,,,
2019,AIAN,33.64486,unhoused,0.074766
2020,AIAN,21.00000,unhoused,0.059155


#### The Census Bureau estimates that there are roughly 211 indigenous residents in the city of Boulder overall:

In [4]:
df_census[df_census.race == 'AIAN']['population']

3    211.346
Name: population, dtype: float64

#### Now, we can estimate the fraction of boulder's indigenous residents who are unhoused:
* at least 15.9 percent in 2019
* at least 9.9 percent in 2020

In [5]:
# number of unhoused aian residents divided by total number of aian residents, for 2019 and 2020
frac_unhoused_aian = df_pit[df_pit.race == 'AIAN']['population'] / df_census[df_census.race == 'AIAN']['population'].iloc[0]
frac_unhoused_aian

year
2019    0.159193
2020    0.099363
Name: population, dtype: float64

# What fraction of boulder's white residents are unhoused?
* We can use the same data sets and methodology to make this calculation as before

#### lower bound on the number of boulder's unhoused white residents for both years
* at least 307 in 2019
* at least 251 in 2020

In [6]:
df_pit[df_pit.race == 'White']

,race,population,type,frac
year,,,,
2019,White,307.009346,unhoused,0.682243
2020,White,251.000000,unhoused,0.707042


#### The Census Bureau estimates that there are roughly 92,358 white residents in the city of Boulder overall:

In [7]:
df_census[df_census.race == 'White']['population']

0    92358.202
Name: population, dtype: float64

#### Now, we can estimate the fraction of boulder's white residents who are unhoused:
* at least 0.33 percent in 2019
* at least 0.27 percent in 2020

In [8]:
# number of unhoused white residents divided by total number of white residents, for 2019 and 2020
frac_unhoused_white = df_pit[df_pit.race == 'White']['population'] / df_census[df_census.race == 'White']['population'].iloc[0]
frac_unhoused_white

year
2019    0.003324
2020    0.002718
Name: population, dtype: float64

# Using these estimates, we find that <font color='red'>Boulder's indigenous residents are on average 42.2x more likely to be unhoused than Boulder's white residents</font>. 
* 47.9x more likely in 2019
* 36.6x more likely in 2020

In [9]:
frac_unhoused_aian / frac_unhoused_white

year
2019    47.890411
2020    36.561753
Name: population, dtype: float64

In [10]:
(frac_unhoused_aian / frac_unhoused_white).mean()

42.22608197347596